# Using Ragas to Evaluate a RAG Application built with LangChain and LangGraph

In the following notebook, we'll be looking at how [Ragas](https://github.com/explodinggradients/ragas) can be helpful in a number of ways when looking to evaluate your RAG applications!

While this example is rooted in LangChain/LangGraph - Ragas is framework agnostic (you don't even need to be using a framework!).

- 🤝 Breakout Room #1
  1. Task 1: Installing Required Libraries
  2. Task 2: Set Environment Variables
  3. Task 3: Synthetic Dataset Generation for Evaluation using Ragas
  4. Task 4: Evaluating our Pipeline with Ragas
  5. Task 6: Making Adjustments and Re-Evaluating

But first! Let's set some dependencies!

## Dependencies and API Keys:

We'll also need to provide our API keys.

First, OpenAI's for our LLM/embedding model combination!

In [ ]:
# Configure Ragas Authentication
# Choose one of the options below:

# Option 1: Use Ragas Cloud (requires token from https://cloud.ragas.io/)
# os.environ["RAGAS_APP_TOKEN"] = getpass("Please enter your Ragas App Token!")

# Option 2: Disable cloud features to avoid authentication
os.environ["RAGAS_DO_NOT_TRACK"] = "true"

print("Ragas authentication configured")


In [1]:
import os
from getpass import getpass
os.environ["OPENAI_API_KEY"] = getpass("Please enter your OpenAI API key!")

## Generating Synthetic Test Data

We wil be using Ragas to build out a set of synthetic test questions, references, and reference contexts. This is useful because it will allow us to find out how our system is performing.

> NOTE: Ragas is best suited for finding *directional* changes in your LLM-based systems. The absolute scores aren't comparable in a vacuum.

### Data Preparation

We'll prepare our data - which should hopefull be familiar at this point since it's our Loan Data use-case!

Next, let's load our data into a familiar LangChain format using the `DirectoryLoader`.

In [44]:
from langchain_community.document_loaders import DirectoryLoader
from langchain_community.document_loaders import PyMuPDFLoader


path = "data/"
loader = DirectoryLoader(path, glob="*.pdf", loader_cls=PyMuPDFLoader)
docs = loader.load()

### Knowledge Graph Based Synthetic Generation

Ragas uses a knowledge graph based approach to create data. This is extremely useful as it allows us to create complex queries rather simply. The additional testset complexity allows us to evaluate larger problems more effectively, as systems tend to be very strong on simple evaluation tasks.

Let's start by defining our `generator_llm` (which will generate our questions, summaries, and more), and our `generator_embeddings` which will be useful in building our graph.

### Abstracted SDG

The above method is the full process - but we can shortcut that using the provided abstractions!

This will generate our knowledge graph under the hood, and will - from there - generate our personas and scenarios to construct our queries.



In [45]:
from ragas.llms import LangchainLLMWrapper
from ragas.embeddings import LangchainEmbeddingsWrapper
from langchain_openai import ChatOpenAI
from langchain_openai import OpenAIEmbeddings
generator_llm = LangchainLLMWrapper(ChatOpenAI(model="gpt-4.1"))
generator_embeddings = LangchainEmbeddingsWrapper(OpenAIEmbeddings())

In [46]:
from ragas.testset import TestsetGenerator

generator = TestsetGenerator(llm=generator_llm, embedding_model=generator_embeddings)
dataset = generator.generate_with_langchain_docs(docs[:20], testset_size=10)

Applying HeadlinesExtractor:   0%|          | 0/17 [00:00<?, ?it/s]

Applying HeadlineSplitter:   0%|          | 0/20 [00:00<?, ?it/s]

unable to apply transformation: 'headlines' property not found in this node
unable to apply transformation: 'headlines' property not found in this node
unable to apply transformation: 'headlines' property not found in this node


Applying SummaryExtractor:   0%|          | 0/31 [00:00<?, ?it/s]

Property 'summary' already exists in node '72e71b'. Skipping!
Property 'summary' already exists in node 'f7842f'. Skipping!
Property 'summary' already exists in node '67bf4c'. Skipping!
Property 'summary' already exists in node '73400b'. Skipping!
Property 'summary' already exists in node 'b98424'. Skipping!
Property 'summary' already exists in node 'efb1b8'. Skipping!
Property 'summary' already exists in node 'a012fb'. Skipping!
Property 'summary' already exists in node 'caad3e'. Skipping!
Property 'summary' already exists in node 'b07b90'. Skipping!
Property 'summary' already exists in node '73b2ba'. Skipping!
Property 'summary' already exists in node '428044'. Skipping!
Property 'summary' already exists in node 'b5fe53'. Skipping!
Property 'summary' already exists in node 'f2ae6a'. Skipping!
Property 'summary' already exists in node '27b8a5'. Skipping!


...

Property 'summary_embedding' already exists in node '73400b'. Skipping!
Property 'summary_embedding' already exists in node '72e71b'. Skipping!
Property 'summary_embedding' already exists in node 'b98424'. Skipping!
Property 'summary_embedding' already exists in node '27b8a5'. Skipping!
Property 'summary_embedding' already exists in node 'caad3e'. Skipping!
Property 'summary_embedding' already exists in node '67bf4c'. Skipping!
Property 'summary_embedding' already exists in node 'a012fb'. Skipping!
Property 'summary_embedding' already exists in node 'f7842f'. Skipping!
Property 'summary_embedding' already exists in node 'efb1b8'. Skipping!
Property 'summary_embedding' already exists in node 'b5fe53'. Skipping!
Property 'summary_embedding' already exists in node 'b07b90'. Skipping!
Property 'summary_embedding' already exists in node '73b2ba'. Skipping!
Property 'summary_embedding' already exists in node 'f2ae6a'. Skipping!
Property 'summary_embedding' already exists in node '428044'. Sk

Applying [CosineSimilarityBuilder, OverlapScoreBuilder]:   0%|          | 0/2 [00:00<?, ?it/s]

Generating personas:   0%|          | 0/3 [00:00<?, ?it/s]

Generating Scenarios:   0%|          | 0/3 [00:00<?, ?it/s]

Generating Samples:   0%|          | 0/12 [00:00<?, ?it/s]

In [5]:
dataset.to_pandas()

,user_input,reference_contexts,reference,synthesizer_name
0,what is BBAY 1 for direct loan annual loan lim...,"[non-term (includes clock-hour calendars), or ...",BBAY 1 is one of the options that can be used ...,single_hop_specifc_query_synthesizer
1,Are osteopathic clinical periods eligible to b...,[Inclusion of Clinical Work in a Standard Term...,"Yes, required clinical periods in osteopathic ...",single_hop_specifc_query_synthesizer
2,"How FWS work with payment periods, it same lik...",[Non-Term Characteristics A program that measu...,The payment period is applicable to all Title ...,single_hop_specifc_query_synthesizer
3,Where can I find information on Direct Loan an...,[both the credit or clock hours and the weeks ...,For information on Direct Loan annual loan lim...,single_hop_specifc_query_synthesizer
4,Under what conditions can a practicum or clini...,[<1-hop>\n\nInclusion of Clinical Work in a St...,A practicum or clinical experience required fo...,multi_hop_abstract_query_synthesizer
5,If a medical or education program requires all...,[<1-hop>\n\nInclusion of Clinical Work in a St...,A medical or education program that requires a...,multi_hop_abstract_query_synthesizer
6,If a medical or education program requires all...,[<1-hop>\n\nInclusion of Clinical Work in a St...,A medical or education program that requires a...,multi_hop_abstract_query_synthesizer
7,If a medical program requires a practicum or c...,[<1-hop>\n\nInclusion of Clinical Work in a St...,If all students in a medical program are requi...,multi_hop_abstract_query_synthesizer
8,What are the requirements for Direct Loan disb...,[<1-hop>\n\nDisbursement Timing in Subscriptio...,"In subscription-based programs, for the first ...",multi_hop_specific_query_synthesizer
9,How do the requirements for Direct Loan disbur...,[<1-hop>\n\nnon-term (includes clock-hour cale...,"In subscription-based programs, for the first ...",multi_hop_specific_query_synthesizer


## LangChain RAG

Now we'll construct our LangChain RAG, which we will be evaluating using the above created test data!

### R - Retrieval

Let's start with building our retrieval pipeline, which will involve loading the same data we used to create our synthetic test set above.

> NOTE: We need to use the same data - as our test set is specifically designed for this data.

In [6]:
path = "data/"
loader = DirectoryLoader(path, glob="*.pdf", loader_cls=PyMuPDFLoader)
docs = loader.load()

Now that we have our data loaded, let's split it into chunks!

In [7]:
from langchain.text_splitter import RecursiveCharacterTextSplitter

text_splitter = RecursiveCharacterTextSplitter(chunk_size=1000, chunk_overlap=200)
split_documents = text_splitter.split_documents(docs)
len(split_documents)

1102

#### ❓ Question: 

What is the purpose of the `chunk_overlap` parameter in the `RecursiveCharacterTextSplitter`?

##### ✅ Answer:



The chunk_overlap parameter serves several critical purposes in document chunking:

**Context Preservation**: Maintains continuity between adjacent chunks by including overlapping text, preventing loss of context at chunk boundaries

**Semantic Coherence**: Ensures that sentences or concepts that span across chunk boundaries remain accessible in both chunks

**Improved Retrieval**: Increases the likelihood that relevant information is captured during retrieval, even if a query relates to content near chunk boundaries

**Reduced Information Loss**: Prevents important relationships or context from being lost when documents are split into smaller pieces

**Better Embedding Quality**: Overlapping content provides richer context for embedding generation, leading to more accurate semantic representations

Next up, we'll need to provide an embedding model that we can use to construct our vector store.

In [8]:
from langchain_openai import OpenAIEmbeddings

embeddings = OpenAIEmbeddings(model="text-embedding-3-small")

Now we can build our in memory QDrant vector store.

In [9]:
from langchain_qdrant import QdrantVectorStore
from qdrant_client import QdrantClient
from qdrant_client.http.models import Distance, VectorParams

client = QdrantClient(":memory:")

client.create_collection(
    collection_name="loan_data",
    vectors_config=VectorParams(size=1536, distance=Distance.COSINE),
)

vector_store = QdrantVectorStore(
    client=client,
    collection_name="loan_data",
    embedding=embeddings,
)

We can now add our documents to our vector store.

In [10]:
_ = vector_store.add_documents(documents=split_documents)

Let's define our retriever.

In [19]:
retriever = vector_store.as_retriever(search_kwargs={"k": 5})

Now we can produce a node for retrieval!

In [20]:
def retrieve(state):
  retrieved_docs = retriever.invoke(state["question"])
  return {"context" : retrieved_docs}

### Augmented

Let's create a simple RAG prompt!

In [21]:
from langchain.prompts import ChatPromptTemplate

RAG_PROMPT = """\
You are a helpful assistant who answers questions based on provided context. You must only use the provided context, and cannot use your own knowledge.

### Question
{question}

### Context
{context}
"""

rag_prompt = ChatPromptTemplate.from_template(RAG_PROMPT)

### Generation

We'll also need an LLM to generate responses - we'll use `gpt-4o-nano` to avoid using the same model as our judge model.

In [22]:
from langchain_openai import ChatOpenAI

llm = ChatOpenAI(model="gpt-4.1-nano")

Then we can create a `generate` node!

In [23]:
def generate(state):
  docs_content = "\n\n".join(doc.page_content for doc in state["context"])
  messages = rag_prompt.format_messages(question=state["question"], context=docs_content)
  response = llm.invoke(messages)
  return {"response" : response.content}

### Building RAG Graph with LangGraph

Let's create some state for our LangGraph RAG graph!

In [24]:
from langgraph.graph import START, StateGraph
from typing_extensions import List, TypedDict
from langchain_core.documents import Document

class State(TypedDict):
  question: str
  context: List[Document]
  response: str

Now we can build our simple graph!

> NOTE: We're using `add_sequence` since we will always move from retrieval to generation. This is essentially building a chain in LangGraph.

In [25]:
graph_builder = StateGraph(State).add_sequence([retrieve, generate])
graph_builder.add_edge(START, "retrieve")
graph = graph_builder.compile()

Let's do a test to make sure it's doing what we'd expect.

In [26]:
response = graph.invoke({"question" : "What are the different kinds of loans?"})

In [27]:
response["response"]

'Based on the provided context, the different kinds of loans mentioned are:\n\n1. Direct Loan\n2. Direct Unsubsidized Loan\n\nThe context specifically references these types when discussing loan counseling, eligibility, and repayment options.'

## Evaluating the App with Ragas

Now we can finally do our evaluation!

We'll start by running the queries we generated usign SDG above through our application to get context and responses.

In [28]:
for test_row in dataset:
  response = graph.invoke({"question" : test_row.eval_sample.user_input})
  test_row.eval_sample.response = response["response"]
  test_row.eval_sample.retrieved_contexts = [context.page_content for context in response["context"]]

In [29]:
dataset.samples[0].eval_sample.response

"BBAY 1 for a direct loan refers to a specific academic period used to monitor and reset a student's annual loan limit eligibility. When BBAY 1 has elapsed, a student regains eligibility for a new annual loan limit, allowing them to receive additional loans. Alternating an SAY (Scheduled Academic Year) with BBAY 1 can enable a student to receive a loan sooner than with an SAY alone, especially in cases where the student wants to access summer loans earlier in the academic year."

Then we can convert that table into a `EvaluationDataset` which will make the process of evaluation smoother.

In [30]:
from ragas import EvaluationDataset

evaluation_dataset = EvaluationDataset.from_pandas(dataset.to_pandas())

We'll need to select a judge model - in this case we're using the same model that was used to generate our Synthetic Data.

In [31]:
from ragas import evaluate
from ragas.llms import LangchainLLMWrapper

evaluator_llm = LangchainLLMWrapper(ChatOpenAI(model="gpt-4.1-mini"))

Next up - we simply evaluate on our desired metrics!

In [32]:
from ragas.metrics import LLMContextRecall, Faithfulness, FactualCorrectness, ResponseRelevancy, ContextEntityRecall, NoiseSensitivity
from ragas import evaluate, RunConfig

custom_run_config = RunConfig(timeout=360)

result = evaluate(
    dataset=evaluation_dataset,
    metrics=[LLMContextRecall(), Faithfulness(), FactualCorrectness(), ResponseRelevancy(), ContextEntityRecall(), NoiseSensitivity()],
    llm=evaluator_llm,
    run_config=custom_run_config
)
result

Evaluating:   0%|          | 0/72 [00:00<?, ?it/s]

Exception raised in Job[65]: TimeoutError()


{'context_recall': 0.8729, 'faithfulness': 0.9335, 'factual_correctness': 0.6300, 'answer_relevancy': 0.9544, 'context_entity_recall': 0.2911, 'noise_sensitivity_relevant': 0.2839}

## Making Adjustments and Re-Evaluating

Now that we've got our baseline - let's make a change and see how the model improves or doesn't improve!

> NOTE: This will be using Cohere's Rerank model - please be sure to [sign-up for an API key!](https://docs.cohere.com/reference/about)

In [33]:
os.environ["COHERE_API_KEY"] = getpass("Please enter your Cohere API key!")


We'll first set our retriever to return more documents, which will allow us to take advantage of the reranking.

In [34]:
adjusted_example_retriever = vector_store.as_retriever(search_kwargs={"k": 20})

Reranking, or contextual compression, is a technique that uses a reranker to compress the retrieved documents into a smaller set of documents.

This is essentially a slower, more accurate form of semantic similarity that we use on a smaller subset of our documents.

In [35]:
from langchain.retrievers.contextual_compression import ContextualCompressionRetriever
from langchain_cohere import CohereRerank

def retrieve_adjusted(state):
  compressor = CohereRerank(model="rerank-v3.5")
  compression_retriever = ContextualCompressionRetriever(
    base_compressor=compressor, base_retriever=adjusted_example_retriever, search_kwargs={"k": 5}
  )
  retrieved_docs = compression_retriever.invoke(state["question"])
  return {"context" : retrieved_docs}

We can simply rebuild our graph with the new retriever!

In [36]:
class AdjustedState(TypedDict):
  question: str
  context: List[Document]
  response: str

adjusted_graph_builder = StateGraph(AdjustedState).add_sequence([retrieve_adjusted, generate])
adjusted_graph_builder.add_edge(START, "retrieve_adjusted")
adjusted_graph = adjusted_graph_builder.compile()

In [37]:
response = adjusted_graph.invoke({"question" : "What are the different kinds of loans?"})
response["response"]

'The context mentions two specific types of loans: Federal PLUS Loans and Direct Loans (which include Direct Subsidized Loans and Direct Unsubsidized Loans). However, it does not explicitly list all the different kinds of loans. Based on the information provided, the different kinds of loans include:\n\n1. Federal PLUS Loans (including student Direct PLUS Loans)\n2. Direct Subsidized Loans\n3. Direct Unsubsidized Loans\n\nAdditionally, there are references to loans made under the Federal Family Education Loan (FFEL) Program before it ended, which indicates there were other FFEL Program loans, but they are not explicitly named in the provided text.\n\nTherefore, the different kinds of loans mentioned are:\n- Federal PLUS Loans\n- Direct Subsidized Loans\n- Direct Unsubsidized Loans'

In [38]:
import time
import copy

rerank_dataset = copy.deepcopy(dataset)

for test_row in rerank_dataset:
  response = adjusted_graph.invoke({"question" : test_row.eval_sample.user_input})
  test_row.eval_sample.response = response["response"]
  test_row.eval_sample.retrieved_contexts = [context.page_content for context in response["context"]]
  time.sleep(2) # To try to avoid rate limiting.

In [40]:
rerank_dataset.samples[0].eval_sample.response

"BBAY 1 for direct loan annual loan limit progression is an alternative to the Scheduled Academic Year (SAY) used to monitor a student's annual loan limit. Specifically, for credit-hour programs with an SAY, BBAY 1 must include the same number of terms as the SAY (excluding any summer trailer or header). The BBAY 1 period can include terms the student does not attend if the student could have enrolled at least half-time in those terms, but it must start with a term in which the student was enrolled."

In [41]:
rerank_evaluation_dataset = EvaluationDataset.from_pandas(rerank_dataset.to_pandas())

In [42]:
result = evaluate(
    dataset=rerank_evaluation_dataset,
    metrics=[LLMContextRecall(), Faithfulness(), FactualCorrectness(), ResponseRelevancy(), ContextEntityRecall(), NoiseSensitivity()],
    llm=evaluator_llm,
    run_config=custom_run_config
)
result

Evaluating:   0%|          | 0/72 [00:00<?, ?it/s]

{'context_recall': 0.8347, 'faithfulness': 0.9581, 'factual_correctness': 0.7033, 'answer_relevancy': 0.8763, 'context_entity_recall': 0.4972, 'noise_sensitivity_relevant': 0.2600}

#### ❓ Question: 

Which system performed better, on what metrics, and why?

##### ✅ Answer:

✅ Reranking Victories:

Factual Correctness: +7.33% (63.00% → 70.33%) - Significant improvement!
Context Entity Recall: +20.61% (29.11% → 49.72%) - Massive improvement!
Faithfulness: +2.46% (93.35% → 95.81%) - Small but meaningful gain
Noise Sensitivity: -2.39% (lower is better) - Better focus

⚠️ Baseline Advantages:

Answer Relevancy: -7.81% (95.44% → 87.63%) - Notable decrease
Context Recall: -3.82% (87.29% → 83.47%) - Some context loss

🔍 What This Tells Us:

Reranking successfully improved content quality - especially factual accuracy and entity coverage
The trade-off was retrieval completeness - some relevant context was lost in compression
Quality vs. Quantity tension - classic information retrieval challenge

This is actually a success story for reranking - it achieved the goal of improving factual correctness while maintaining high faithfulness. The decision between systems would depend on whether you prioritize accuracy (reranking) or comprehensive relevancy (baseline).